## Introduction to Hub Discussions

Hub Discussions was the result of many requests for in-app communication and collboration expressed by many customers of ArcGIS Hub. Hub Discussions will help to support integrated collaboration between users, organizations, and staff.

To start using Hub Discussions, you must first connect to Hub using the Hub object from the arcgishub module.

In [59]:
from arcgishub import hub

To create the Hub object, we pass in a url and our login credentials:

In [60]:
my_hub = hub.Hub(url='https://prod-pre-hub.maps.arcgis.com', username='prod_pre_hub_admin', password='prod1hubtest')

### Hub Discussions Overview

Before we can use the Hub object that we created above into Hub Discussions, it is important to understand the high level architecture of Discussions. 

Channels and Posts are the central models to the Discussions system. Currently, Channels and Posts are related by a simple one-to-many relationship, where a channel may have many posts and a post belongs to a single channel. A Post can have zero or many Reactions, which represent a user's elicited response to a post. Posts can also relate to other posts as replies.

#### Channels

A Channel represents a unique platform sharing configuration, comprised of a distinct set of values for access, groups, and orgs. Channels are related to an ArcGIS Online user through their portal organization and group memberships, and subsequently the Channel model tracks closely with the platform sharing ACL.

There are 4 varieties of channels that can be created:
- Team Channel - access is private and groups array contains a single platform groupId
- Shared Channel - access is private and there are multiple platform groupIds in the groups array
- Org Channel - access is limited to members of the configured orgs
- Public Channel - Can be viewed by everyone but is configured with the org value of creator

#### Posts
A Post is the primary forum for dialogue between users in a Channel. Posts are organized into "threads" using a self-join: a Post can have many replies, or a Post can belong to a Parent Post. Posts are required to have a body, and optionally a title (perhaps best suited for parent posts). A Post also has a status, which affect the visibility of a post and values such as "pending", "approved", or "rejected" serve as an early mechanism for moderation.

Posts can also relate to platform content such as items, datasets, and groups via the discussion attribute. This attribute is a URI-formatted string. 

Examples:
- *hub://dataset/4ef_1?id=4&attribute=species* encodes that a post was made in the Hub application about dataset 4ef, layer _1, feature 4's species attribute. 
- *urban://item/1b4* encodes that a post was made in Urban about plan 1bf. Perhaps the author of the post has a critique of the plan at large.


To begin using the Hub Discussions Python Module, simply use the discussions object of your hub to access posts and channels. This will also give access to all the different functions available to both entities. Let's look at some examples of the usage.

### Interacting with Channels

#### We can use the search function to get all the channels

In [70]:
my_hub.discussions.channels.search()

[<channel_id:2b3709f2d7884e1aab1972be01f5bd68 access:"private" groups:['2b3709f2d7884e1aab1972be01f5bd68'] creator:qa_pre_hub_admin>,
 <channel_id:3603f8101f5e4c229ce67cf0a3e160ce access:"public" groups:[] creator:qa_pre_hub_admin>,
 <channel_id:b8cbcd94a896457b9df3c9d25ca45c61 access:"org" groups:[] creator:qa_pre_hub_admin>]

#### We can also get a specific channel using it's id

In [62]:
# Get the first channel in the list of channels from our search results. Use that id to retrieve that channel.
single_channel = my_hub.discussions.channels.search()[0]
channel_id = single_channel.id
channel_id

'2b3709f2d7884e1aab1972be01f5bd68'

In [63]:
retrieved_channel = my_hub.discussions.channels.get(channel_id)
retrieved_channel

<channel_id:2b3709f2d7884e1aab1972be01f5bd68 access:"private" groups:['2b3709f2d7884e1aab1972be01f5bd68'] creator:qa_pre_hub_admin>

#### Creating a New Channel

In order to create a new channel, we need to provide a dictionary of properties to create the channel. As explained before, this will be the configurations for the channel. 

To create a channel, there are two **required** arguments that must be specific in the configuration:
- access: this is the platform level access. Can be "org", "private", or "public".
- groups: required for private or org access configs, will be a array of platform groupIds

There's also a handful of **optional** properties that you can configure when creating a channel:
- allowReply: determines whether replies can be made to posts (boolean)
- allowAnonymous: for public channels, determines if unauthenticated users can make posts (boolean)
- softDelete: delete operations flag posts as deleted rather than an SQL query to delete (boolean)
- defaultPostStatus: initial status applied to posts (approved, pending, rejected, deleted, and hidden)
- allowReaction: determines whether reactions can be created for posts in channel (boolean)
- allowedReactions: reactions that can be made for posts (example: "thumbs_up", etc.)
- blockWords: Not Yet Implemented. Will be used for words/phrases that will automatically be moderated

With that out of the way, let's try to create our own channel


First, we create an object that will let us set the properties of the channel we wish to create.

In [68]:
channel_properties = {
    "access": "org",
    "groups": ["T5cZDlfUaBpDnk6P"]
}

Now, let's create the channel using the property object we just made

In [69]:
new_channel = my_hub.discussions.channels.add(channel_properties)
new_channel

<channel_id:b8cbcd94a896457b9df3c9d25ca45c61 access:"org" groups:[] creator:qa_pre_hub_admin>

Just like that we have our new channel!

#### Getting the different properties
Now that our channel has been created, let's try to get each of it's properties,

In [11]:
new_channel.id

'7e4f811e785242759cac22b7c5a30d54'

In [12]:
new_channel.allowReply

True

In [13]:
new_channel.allowAnonymous

False

In [14]:
new_channel.softDelete

True

In [15]:
new_channel.defaultPostStatus

'approved'

In [16]:
new_channel.allowReaction

True

In [17]:
new_channel.access, new_channel.orgs

('org', ['T5cZDlfUaBpDnk6P'])

In [18]:
new_channel.groups

[]

In [19]:
new_channel.creator, new_channel.editor

('qa_pre_hub_admin', 'qa_pre_hub_admin')

In [20]:
new_channel.created, new_channel.updated

('2021-10-15T15:07:44.522Z', '2021-10-15T15:07:44.522Z')

#### Updating a Channel
Now that you've seen the different properties, let's try to update something on that channel. Let's update the allowedReactions property to allow posts to have the thumbs up reaction.

In [21]:
updated_channel = new_channel.update(allowedReactions=["thumbs_up"])
updated_channel

<channel_id:7e4f811e785242759cac22b7c5a30d54 access:"org" groups:[] creator:qa_pre_hub_admin>

#### Deleting a Channel
It's quite simple to delete a channel. Just called the delete function on the object itself.

In [22]:
deleted = new_channel.delete()
deleted

True

### Interacting with Posts
Now that we've gone over Channels and their functionalities, we can finally look into posts. The posts functionality is very similar to the channels that we saw above and interact in almost the same way. We can access the posts attribute of discussinos to play with all of it's functionality.

#### Search for all posts

In [23]:
all_posts = my_hub.discussions.posts.search(max_posts=2)
all_posts

[<title:"None" creator:qa_pre_hub_admin created:2021-10-15T14:59:46.166Z>,
 <title:"None" creator:qa_pre_hub_admin created:2021-10-15T15:02:02.976Z>]

#### Retrieve a specific post by ID

In [29]:
# Get the first post in the list of posts from our search results. Use that id to retrieve that post.
single_post = my_hub.discussions.posts.search()[0]
post_id = single_post.id
post_id

'a6d407f6660b4bdba0760662c771d3b0'

In [30]:
retrieved_post = my_hub.discussions.posts.get(post_id)
retrieved_post

<title:"main title" creator:qa_pre_hub_admin created:2021-10-15T15:15:49.742Z>

#### Creating a New Post

Just like how we had to pass in a dictionary of properties to create a channel, we must do the same for creating a post. However, the requirements for the properties are a bit different for posts.

To create a post, there are multiple required properties. No matter what, the body is required. This will be the primary text content of the post. Aside from that, youw ill either need to provide **either** channelId **OR** access/groups (yes, both access and groups).

So you will either need to provide at minimum the following two options:

```
{
   "channelId": "channelId12345",
   "body": "hello there
}
```
**OR**
```
{
   "access": "private",
   "groups": ["groupId12345"],
   "body": "hello there
}
``` 

Aside from the required posts, you can also pass add in other optional properties when creating a post. Some of these optional properties include:
- title: title of the post, usually for initial post of discussion (string)
- discussion: valid discussion URI showing post's relation to platform content (string)
- geometry: geometry property of GeoJSON spec. spec required WGS84 (string)
- appInfo: generic field for application specific notes. Urban uses this to encode a "topic" to posts. (string)
- parentId: used for creating a reply, points to the parent post (string)

In [31]:
post_properties = {
            "access": "private",
            "groups": ["2b3709f2d7884e1aab1972be01f5bd68"],
            "discussion": "hub://item/uuid",
            "title": "this is my title",
            "body": "hello there"
        }

Now, let's create the channel using the property object we just made

In [36]:
new_post = my_hub.discussions.posts.add(post_properties)
new_post

<title:"this is my title" creator:qa_pre_hub_admin created:2021-10-15T15:17:07.972Z>

#### Getting the post's different properties

In [37]:
new_post.id, new_post.channelId

('e081afc4c21249c3956c32da7c99e300', '2b3709f2d7884e1aab1972be01f5bd68')

In [38]:
new_post.title, new_post.body

('this is my title', 'hello there')

In [39]:
new_post.discussion

'hub://item/uuid'

In [40]:
new_post.creator, new_post.editor

('qa_pre_hub_admin', 'qa_pre_hub_admin')

In [41]:
new_post.created, new_post.updated

('2021-10-15T15:17:07.972Z', '2021-10-15T15:17:07.972Z')

In [42]:
# only has value if it is a reply post
new_post.parentId

#### updating a post

In [43]:
updated_post = new_post.update(title='my updated title')
updated_post

<title:"my updated title" creator:qa_pre_hub_admin created:2021-10-15T15:17:07.972Z>

#### adding reactions to a post
You can add reactions on a post object directly. The following reaction values are allowed on a post:
- thumbs_up
- thumbs_down
- thinking
- heart
- one_hundred
- sad
- laughing
- surprised

In [44]:
addReaction = updated_post.add_reaction("thumbs_up")
addReaction

True

#### deleting reactions
You can delete a reaction by providingi the id of the reaction that was added.

In [45]:
deleteReaction = updated_post.delete_reaction("9900b69564fd49119a100b3728620d95")
deleteReaction

True

#### deleting a post
Finally, it's simple to delete a post. just called the delete function on the post you would like to delete.

In [46]:
delete = updated_post.delete()

### Conclusion and Future Ideas

That wraps up the majority of the features provided to python developers to use the Discussion API. In the future, there will be additional features being added such as moderation tools including a Toxic Comment Classifier that can auto moderate posts to a certain level. There will also be more channel configurations as mentioned above such as shared channels.
